build standard ans of malware family

In [172]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import os, shutil

In [115]:
ori_df = pd.read_csv("./data/Hooklog/VTRepo_wn_AAMerge.csv")
ori_df = ori_df.fillna("no_name")
ori_df

/home/leoqaz12/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (37,55,58,66,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,ALYac,AVG,AVware,Acronis,Ad-Aware,AegisLab,Agnitum,AhnLab-V3,Alibaba,...,Webroot,WhiteArmor,Yandex,Zillya,ZoneAlarm,Zoner,ahnlab,eGambit,eSafe,nProtect
0,b048ab49d6bceac3161a369ff83679e870ddcd70a2b252...,no_name,no_name,downloadmr,no_name,no_name,no_name,solimba,no_name,no_name,...,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name
1,164d8ccd062f92f1b12ef1d6c3ba282ee28441bbd0bc3c...,no_name,no_name,no_name,no_name,no_name,no_name,uysyt,no_name,no_name,...,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name
2,013b72d5c6d8b5b2139ef2edef18568dfe01d7bf019c37...,no_name,filerepma,no_name,no_name,lmkfr,no_name,no_name,refroso,no_name,...,no_name,no_name,vbinject,no_name,no_name,no_name,no_name,unsafe,no_name,no_name
3,7f25186ea379f37cf55bd003855ed665fdb79ea153c52e...,no_name,fakeav,winwebsec,no_name,no_name,no_name,kryptik,fakeav,no_name,...,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,mbro
4,7bbc7153effc4aa7dfe232858012eff84b20730626afec...,symmi,malob,winwebsec,no_name,symmi,fakeav,no_name,fakeav,no_name,...,no_name,highconfidence,kryptik,fakeav,no_name,no_name,no_name,no_name,no_name,smartfortress
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21719,1e5154aeb5720f0b6e7f14704b597bd8838027db3ad120...,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,...,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name
21720,065d84aa4cc7932eb7c6bf397324f65b97abf1d94b3c68...,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,...,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name
21721,b10c0d59b4bd1a0f67ab104ac3b136170a763f32b0deb0...,no_name,cjkd,no_name,no_name,no_name,no_name,sytro,no_name,no_name,...,no_name,no_name,no_name,sytro,no_name,no_name,no_name,no_name,no_name,no_name
21722,0aed09036edb4e086cda70ff48ba1121ee7b0f81bdc180...,no_name,no_name,onekit,no_name,no_name,no_name,no_name,no_name,no_name,...,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name,no_name


In [180]:
ori_df[ori_df['Unnamed: 0'] == 'fed55bd9dd509d59d240700404c38d20de75e1e72218c83e473e6833cb6eb5b3']

,Unnamed: 0,ALYac,AVG,AVware,Acronis,Ad-Aware,AegisLab,Agnitum,AhnLab-V3,Alibaba,...,Webroot,WhiteArmor,Yandex,Zillya,ZoneAlarm,Zoner,ahnlab,eGambit,eSafe,nProtect
16935,fed55bd9dd509d59d240700404c38d20de75e1e72218c8...,browsefox,no_name,browsefox,no_name,browsefox,yotoon,no_name,browsefox,no_name,...,no_name,no_name,browsefox,browsefo,no_name,no_name,no_name,no_name,no_name,no_name


In [116]:
ans_df = pd.DataFrame(columns=["AV_hash","fam_name"]) #原始query的hash
for i in tqdm(range(len(ori_df))):
    ori_li = ori_df.iloc[i].values.tolist()
    hash_id = ori_li[0]
    del ori_li[0]
    name_li = [x for x in ori_li if x != "no_name"]
    name_count = Counter(name_li).most_common()
    try:
        first_num = name_count[0][1]
    except IndexError :
        continue
    try:
        second_num = name_count[1][1]
    except IndexError:
        second_num = int((len(ori_df.columns) - 1)/6) #all AV vendors number 
    if first_num > 2*second_num: #1/3 of AV vender agree if there's no second_num
        temp = [hash_id,name_count[0][0]]
        temp = pd.Series(temp, index = ans_df.columns)
        ans_df = ans_df.append(temp, ignore_index=True)
ans_df

100%|██████████| 21724/21724 [00:10<00:00, 2154.42it/s]


,AV_hash,fam_name
0,013b72d5c6d8b5b2139ef2edef18568dfe01d7bf019c37...,lmkfr
1,3c20377722db0358693892fc31e312f5cb6bf260b6dec2...,fakeav
2,137a615072cd00d1e2d411eda289b0c89490c50ffc87e3...,kazy
3,1ed036a08cbdb09f01d7b58b21e45252a99efe83a36541...,autoit
4,a9a8bcacbfb36307990f3c8cabce00121585926e9e0eee...,ramnit
...,...,...
8893,0c078509b4665aa05b336cd67107ca6aa799a457a75d95...,hotbar
8894,13dc569cfe97fa8d0ec19c57cacde67439f8c3b3aaca80...,sality
8895,0e2ebc55e26f31678e1229e972538398783ead646c2870...,soft
8896,b10c0d59b4bd1a0f67ab104ac3b136170a763f32b0deb0...,sytro


In [179]:
ans_df[ans_df['AV_hash'] == 'fca30fe156387a28e5aad03ec4fa945e828f9abd2760b074ce535f803f26185a']

,AV_hash,fam_name
4609,fca30fe156387a28e5aad03ec4fa945e828f9abd2760b0...,sytro


In [130]:
hkl_dir = './data/Hooklog/trace_original_all_param/'
all_hkls_li = next(os.walk(hkl_dir))[2]
print(len(all_hkls_li))

20002


In [134]:
ans2_df = pd.DataFrame(columns=["pid","fam_name"]) #成功有錄製的hash
ans3_df = pd.DataFrame(columns=["success_hash","fam_name"])
processed_hash = []
needed_hash_li = set(ans_df.AV_hash.tolist())
for hkl in tqdm(all_hkls_li):
    hk_hash = hkl.split("_")[0]
    if hk_hash in needed_hash_li:
        temp = [hkl,ans_df[ans_df.AV_hash == hk_hash]['fam_name'].values[0]]
        temp = pd.Series(temp, index = ans2_df.columns)
        ans2_df = ans2_df.append(temp , ignore_index=True)
        if hk_hash not in processed_hash:
            temp = [hk_hash,ans_df[ans_df.AV_hash == hk_hash]['fam_name'].values[0]]
            temp = pd.Series(temp, index= ans3_df.columns)
            ans3_df = ans3_df.append(temp, ignore_index=True)
            processed_hash.append(hk_hash)
ans3_df     #10728 processes

100%|██████████| 20002/20002 [00:29<00:00, 677.88it/s]


,success_hash,fam_name
0,00cefcdc8c4e6ff6c9ef576f99b08ba6ed376b17ed6179...,loadmoney
1,07bf39e5c4946a134bae899dd2b9a6245a190b0be8b90b...,sality
2,0db5909eead380b4eef915207966dc4ff04ef5d2460254...,solimba
3,7135d7e745f587fc247cb75cbcb17da6516ac4e9d745a2...,picsys
4,149917e290fd3e71a9a297463efcb1fd39c9299ab149be...,vilsel
...,...,...
6102,0f79282e4d7d251505460f769336a9550c34f5bf41534d...,mediafinder
6103,3276485405694bb5d8ad3461b872638556871c60e01fa3...,ebgx
6104,150662a8e42d999a1ee28fbb80e7de7adbc7d1ec0ed295...,virut
6105,7161ec9de6483cab8bc32abcae4f15563f3ee32dec8116...,delf


In [135]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [136]:
statistics = ans_df.fam_name.tolist()
statistics_pd = pd.DataFrame.from_dict(dict(Counter(statistics)),columns=['samples_num'],orient='index')
print("total fam_number:",len(statistics_pd))
basic_statistics(statistics_pd.samples_num.tolist())

total fam_number: 661


,length
mean,13.461422
std,51.716559
mode,1.000000
min,1.000000
q1,1.000000
median,1.000000
q3,4.000000
max,677.000000
iqr,3.000000
outlier,8.500000


In [142]:
statistics2 = ans2_df.fam_name.tolist()
statistics2_pd = pd.DataFrame.from_dict(dict(Counter(statistics2)),columns=['processes_num'],orient='index')
print("total fam_number:",len(statistics2_pd))
basic_statistics(statistics2_pd.processes_num.tolist())

total fam_number: 484


,length
mean,22.165289
std,85.565508
mode,1.000000
min,1.000000
q1,1.000000
median,3.000000
q3,8.250000
max,1283.000000
iqr,7.250000
outlier,19.125000


In [143]:
statistics3 = ans3_df.fam_name.tolist()
statistics3_pd = pd.DataFrame.from_dict(dict(Counter(statistics3)),columns=['profiled_samples_num'],orient='index')
print("total fam_number:",len(statistics3_pd))
basic_statistics(statistics3_pd.profiled_samples_num.tolist())

total fam_number: 484


,length
mean,12.617769
std,46.376653
mode,1.000000
min,1.000000
q1,1.000000
median,1.000000
q3,5.000000
max,625.000000
iqr,4.000000
outlier,11.000000


In [148]:
statistics_merge_pd = pd.merge(statistics_pd,statistics3_pd,right_index=True,left_index=True,how='outer')
statistics_merge_pd = pd.merge(statistics_merge_pd, statistics2_pd, right_index=True, left_index=True,how='outer')
statistics_merge_pd = statistics_merge_pd.fillna(0)
statistics_merge_pd = statistics_merge_pd.astype('int')
statistics_merge_pd.to_excel('./results/Hooklog/statistics_fam_dist.xlsx')
statistics_merge_pd

,samples_num,profiled_samples_num,processes_num
aacutid,1,0,0
aanql,1,1,2
aaqx,1,0,0
aayt,1,0,0
acsw,1,0,0
...,...,...,...
zomon,1,0,0
zugo,1,0,0
zusy,72,37,62
zvuzona,2,2,3


In [ ]:
def family_take(se):
    if se infam_names:
        return 'take'
    else:
        return 'not_take'

### pick family

In [168]:
used_df = statistics_merge_pd[(statistics_merge_pd['processes_num']>160) & (statistics_merge_pd['processes_num']<286)]
fam_names = used_df.index.tolist()
print(fam_names, len(fam_names),'families')
def family_take(se):
    if se in fam_names:
        return 'take'
    else: 
        return 'not_take'
ans2_df['take'] = ans2_df.fam_name.map(family_take)
final_df = ans2_df[ans2_df['take'] == 'take']
final_df.reset_index(inplace=True,drop=True)
final_df.drop(['take'],inplace=True,axis=1)
final_df

['browsefox', 'domaiq', 'installcore', 'loadmoney', 'mplug', 'outbrowse', 'parite', 'ramnit', 'sality', 'soft', 'softpulse', 'sytro', 'vilsel', 'zbot'] 14


/home/leoqaz12/.local/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,pid,fam_name
0,00cefcdc8c4e6ff6c9ef576f99b08ba6ed376b17ed6179...,loadmoney
1,07bf39e5c4946a134bae899dd2b9a6245a190b0be8b90b...,sality
2,149917e290fd3e71a9a297463efcb1fd39c9299ab149be...,vilsel
3,536500494566979b5ab0311e36d42018b7286b87876952...,ramnit
4,5c8f7968b846c71bfd2f6e7e64222df6c2e9353de335dd...,sytro
...,...,...
3119,b55b5977676707c2f11a01cacefbdabe560c3192d24186...,vilsel
3120,c3ce6341195ad30ada1666eebd258a4942e87f91b7747f...,installcore
3121,cab72334c57f859b73a13ba41e152c72f2b4100622ab0b...,domaiq
3122,11341821c87a8419f2e6885ce129713db3bde0fb16f3ed...,installcore


In [174]:
pick_dir = "./data/Hooklog/trace_picked_fam/"
for i in range(len(final_df)):
    file_name = final_df.loc[i,'pid']
    fam_name = final_df.loc[i,'fam_name']
    out_dir = pick_dir+fam_name+'/'
    if not os.path.exists(out_dir):
        os.makedirs(out_dir,exist_ok=True)
    shutil.copyfile(hkl_dir + file_name,out_dir+file_name)